<a href="https://colab.research.google.com/github/BKousha/FloorPlanSegmentation/blob/main/training_TF_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')
!ls "/content/drive/My Drive"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
'Colab Notebooks'
'Copy of Predictive Maintenance Checklist.gdoc'
 FloorPlan_samples
 model_best_val_loss_var.pkl
 Processed_Floor_Plans
 ShotBot
'SNN_soft shadow network for image compositing.pdf'
 test.txt


In [2]:

! pip install segmentation_models -q
# ! pip install -U segmentation-models==0.2.1


In [3]:
import os
os.environ["SM_FRAMEWORK"] = "tf.keras"

import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dropout, concatenate, UpSampling2D,Conv2DTranspose
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import backend as K

from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.colors import ListedColormap
import matplotlib as mlp


# ! pip install --upgrade keras
import segmentation_models as sm

from tensorflow.keras.saving import get_custom_objects


Segmentation Models: using `tf.keras` framework.


In [4]:
print(tf.__version__)

try: # detect TPUs
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
    print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver() # TPU detection
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
    tpu_resolver = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(tpu_resolver)
    tf.tpu.experimental.initialize_tpu_system(tpu_resolver)
except ValueError: # no TPU found, detect GPUs
    #strategy = tf.distribute.MirroredStrategy() # for GPU or multi-GPU machines
    strategy = tf.distribute.get_strategy() # default strategy that works on CPU and single GPU
    #strategy = tf.distribute.experimental.MultiWorkerMirroredStrategy() # for clusters of multi-GPU machines

try:
  # Check the device being used for training
  device_name = tf.test.gpu_device_name()

  if device_name != '/device:GPU:0':
      device_name = tf.test.tpu_device_name()

  if device_name == '':
      device_name = 'CPU'
except:
  pass
print('Training device:', device_name)

AUTO = tf.data.experimental.AUTOTUNE


2.12.0
Training device: /device:GPU:0


In [5]:
# Define the class mapping for merging
class_names = {
    0: "background",
    1: "room",
    2: "hallway",
    3: "kitchen",
    4: "columns",
    5: "mep",
    6: "windows",
    7: "doors",
    8: "glass_doors",
    9: "bathrooms",
    10: "elevators",
    11: "elevator_doors",
    12: "stairs",
    13: "storage",
    14: "electrical"
}


In [6]:


df=pd.read_csv('/content/drive/MyDrive/Processed_Floor_Plans/smaller_images/pixel_counts.csv')
#df=pd.read_csv('/content/drive/MyDrive/Processed_Floor_Plans/smaller_images/pixel_counts_128_128.csv')

#df
mask_files=df['filename'].values.tolist()
len(mask_files)
image_files = [filename.replace("temp_masks", "temp_images") for filename in mask_files]
image_files

print(len(image_files))
print(len(mask_files))

display(df)


batch_size = 8

num_classes = 14
Number_Category = 14
validation_split=0.1
debug=False

image_size=512
mask_category_list=[7,8]
#mask_category_list=[6]
debug=False
iplot=False



24903
24903


,filename,background,room,hallway,kitchen,columns,mep,windows,doors,glass_doors,bathrooms,elevators,elevator_doors,stairs,storage,electrical
0,/content/drive/MyDrive/Processed_Floor_Plans/s...,0.591652,0.118961,0.007053,0.278271,0.001350,0.002712,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
1,/content/drive/MyDrive/Processed_Floor_Plans/s...,0.752861,0.197372,0.010159,0.025391,0.003078,0.007637,0.000042,0.002728,0.000732,0.000000,0.000000,0.000000,0.000000,0.0,0.0
2,/content/drive/MyDrive/Processed_Floor_Plans/s...,0.367546,0.336494,0.009842,0.012436,0.000931,0.076778,0.195972,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
3,/content/drive/MyDrive/Processed_Floor_Plans/s...,0.450470,0.321926,0.009510,0.008259,0.001698,0.048916,0.159222,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
4,/content/drive/MyDrive/Processed_Floor_Plans/s...,0.543015,0.235577,0.012028,0.171329,0.003754,0.003048,0.031250,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24898,/content/drive/MyDrive/Processed_Floor_Plans/s...,0.614685,0.209835,0.045422,0.008781,0.006222,0.021400,0.003525,0.001793,0.018581,0.015846,0.000668,0.010021,0.043221,0.0,0.0
24899,/content/drive/MyDrive/Processed_Floor_Plans/s...,0.644474,0.136776,0.119335,0.008301,0.003670,0.019718,0.001743,0.001202,0.000488,0.000420,0.000313,0.000366,0.063194,0.0,0.0
24900,/content/drive/MyDrive/Processed_Floor_Plans/s...,0.597298,0.182735,0.085327,0.007359,0.005657,0.020821,0.003559,0.002514,0.032402,0.015125,0.000950,0.016895,0.029358,0.0,0.0
24901,/content/drive/MyDrive/Processed_Floor_Plans/s...,0.552479,0.153652,0.241100,0.006588,0.002357,0.020512,0.001110,0.000496,0.000435,0.000462,0.000206,0.000168,0.020435,0.0,0.0


In [7]:
#image_files[22000],mask_files[22000]

In [8]:


def load_data(image_path, mask_path, mask_category_list, Number_Category=14,debug=False):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_png(image, channels=3)
    image = tf.image.convert_image_dtype(image, tf.float32)

    mask = tf.io.read_file(mask_path)
    mask = tf.image.decode_png(mask, channels=1)
    mask = tf.round(tf.cast(mask, tf.float64) * Number_Category / 255.0)
    condition = tf.math.logical_or(*[tf.equal(mask, val) for val in mask_category_list])
    mask = tf.where(condition, 1, 0)

    #mask = tf.where(tf.equal(mask, mask_category), 1, 0)
    mask = tf.cast(mask, tf.uint8)


    if debug:
        image_filename = tf.strings.split(image_path, os.path.sep)[-1]
        image_filename = tf.py_function(func=lambda x: x.numpy().decode('utf-8'), inp=[image_filename], Tout=tf.string)
        return image, mask, image_filename
    else:
        return image, mask


def augment_data(image, mask):
    # Apply data augmentation to the image and mask
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    #image = tf.image.random_brightness(image, max_delta=0.2)
    #image = tf.image.random_contrast(image, lower=0.8, upper=1.2)

    return image, mask

def create_dataset(image_paths, mask_paths, mask_category_list, batch_size, validation_split=0.2,debug=False,random_state=2023):
    train_image_paths, val_image_paths,train_mask_paths,val_mask_paths = train_test_split(image_paths, mask_paths, test_size=validation_split, random_state=random_state)

    train_dataset = tf.data.Dataset.from_tensor_slices((train_image_paths, train_mask_paths))
    train_dataset = train_dataset.map(lambda x, y: load_data(x, y, mask_category_list=mask_category_list, debug=debug), num_parallel_calls=tf.data.experimental.AUTOTUNE)
    train_dataset = train_dataset.map(augment_data, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    train_dataset = train_dataset.shuffle(buffer_size=1000)
    train_dataset = train_dataset.batch(batch_size)
    train_dataset = train_dataset.prefetch(tf.data.experimental.AUTOTUNE)

    val_dataset = tf.data.Dataset.from_tensor_slices((val_image_paths, val_mask_paths))
    val_dataset = val_dataset.map(lambda x, y: load_data(x, y, mask_category_list=mask_category_list, debug=debug), num_parallel_calls=tf.data.experimental.AUTOTUNE)
    val_dataset = val_dataset.batch(batch_size)
    val_dataset = val_dataset.prefetch(tf.data.experimental.AUTOTUNE)

    return train_dataset, val_dataset

def plot_images_and_masks(dataset, num_batches):
    # Take the specified number of batches from the dataset
    dataset = dataset.take(num_batches)

    for batch in dataset:
        images, masks,files = batch

        # Iterate over each image and mask in the batch
        for i in range(images.shape[0]):
            image = images[i]
            mask = masks[i]

            # Plot image and mask side by side
            fig, axes = plt.subplots(1, 3, figsize=(10, 5))
            axes[0].imshow(image)
            axes[0].set_title('Image')
            axes[0].axis('off')
            axes[1].imshow(mask[:, :, 0], cmap='gray')
            axes[1].set_title('Mask')
            axes[1].axis('off')
            axes[2].imshow(image)
            axes[2].imshow(mask[:, :, 0], alpha=0.5)
            axes[2].axis('off')



            plt.show()


In [9]:
train_dataset, val_dataset = create_dataset(image_files, mask_files, mask_category_list=mask_category_list, batch_size=batch_size, validation_split=validation_split,debug=debug)
num_batches=1
val_dataset = val_dataset.take(num_batches)
for batch in val_dataset:
   images, masks = batch
   print(images.shape,images.dtype,masks.shape,masks.dtype)
   if iplot:
      for i in range(images.shape[0]):
                image = images[i]
                mask = masks[i]
                print(np.unique(mask.numpy()))
                #ifile = files[i]
                #print(ifile)

                # Plot image and mask side by side
                fig, axes = plt.subplots(1, 3, figsize=(10, 5))
                axes[0].imshow(image)
                axes[0].set_title('Image')
                axes[0].axis('off')
                axes[1].imshow(mask[:, :, 0])
                axes[1].set_title('Mask')
                axes[1].axis('off')
                axes[2].imshow(image)
                axes[2].imshow(mask[:, :, 0], alpha=0.5)
                axes[2].axis('off')



                plt.show()


(8, 512, 512, 3) <dtype: 'float32'> (8, 512, 512, 1) <dtype: 'uint8'>


In [10]:
# https://tensorlayer.readthedocs.io/en/latest/_modules/tensorlayer/cost.html#dice_coe
def dice_coe(output, target, axis = None, smooth=1e-10):
    output = tf.dtypes.cast( tf.math.greater(output, 0.5), tf. float32 )
    target = tf.dtypes.cast( tf.math.greater(target, 0.5), tf. float32 )
    inse = tf.reduce_sum(output * target, axis=axis)
    l = tf.reduce_sum(output, axis=axis)
    r = tf.reduce_sum(target, axis=axis)

    dice = (2. * inse + smooth) / (l + r + smooth)
    dice = tf.reduce_mean(dice, name='dice_coe')
    return dice

# https://www.kaggle.com/kool777/training-hubmap-eda-tf-keras-tpu
def tversky(y_true, y_pred, alpha=0.7, beta=0.3, smooth=1):
    y_true_pos = K.flatten(y_true)
    y_pred_pos = K.flatten(y_pred)
    true_pos = K.sum(y_true_pos * y_pred_pos)
    false_neg = K.sum(y_true_pos * (1 - y_pred_pos))
    false_pos = K.sum((1 - y_true_pos) * y_pred_pos)
    return (true_pos + smooth) / (true_pos + alpha * false_neg + beta * false_pos + smooth)
def tversky_loss(y_true, y_pred):
    return 1 - tversky(y_true, y_pred)
def focal_tversky_loss(y_true, y_pred, gamma=0.75):
    tv = tversky(y_true, y_pred)
    return K.pow((1 - tv), gamma)

get_custom_objects().update({"focal_tversky": focal_tversky_loss,'dice_coe':dice_coe})

In [11]:
tf.keras.backend.set_image_data_format('channels_last')
best_model_path=f'/content/drive/MyDrive/Processed_Floor_Plans/smaller_images/best_model_{image_size}.h5'
with strategy.scope():

  model = sm.Unet('efficientnetb0', classes=1, activation='sigmoid',input_shape=(image_size, image_size, 3))
  model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = 0.0001),
                        loss = tf.keras.losses.BinaryCrossentropy(),#'focal_tversky',
                        metrics=[dice_coe,'accuracy'])

  checkpoint = ModelCheckpoint(best_model_path, monitor='val_loss', save_best_only=True, mode='min', verbose=1)
  reduce_lr_callback = ReduceLROnPlateau(monitor='val_loss',factor=0.5,patience=4,verbose=1, min_delta=0.00001)

# Count the number of trainable parameters
trainable_params = int(np.sum([tf.keras.backend.count_params(w) for w in model.trainable_weights]))
print("Trainable Parameters {:,}".format(trainable_params) )

# Count the total number of parameters
total_params = int(np.sum([tf.keras.backend.count_params(w) for w in model.weights]))
print("Total Parameters {:,}".format(total_params) )

Trainable Parameters 10,071,501
Total Parameters 10,115,501


In [ ]:
# Train the model using the data generators
epochs=20
verbose=1

batch_size = batch_size * strategy.num_replicas_in_sync

print("Number of accelerators: ", strategy.num_replicas_in_sync)
print("BATCH_SIZE: ", str(batch_size))

train_dataset, val_dataset = create_dataset(image_files[0:20000], mask_files[0:20000], mask_category_list=mask_category_list, batch_size=batch_size, validation_split=validation_split,debug=False)

K.clear_session()
with strategy.scope():
  history=model.fit(train_dataset, epochs=epochs, validation_data=val_dataset,callbacks=[checkpoint,reduce_lr_callback],verbose=verbose)



Number of accelerators:  1
BATCH_SIZE:  8
Epoch 1/20
225/225 [==============================] - ETA: 0s - loss: 0.3928 - dice_coe: 0.0452 - accuracy: 0.9318
Epoch 1: val_loss improved from inf to 0.31966, saving model to /content/drive/MyDrive/Processed_Floor_Plans/smaller_images/best_model_512.h5
225/225 [==============================] - 381s 1s/step - loss: 0.3928 - dice_coe: 0.0452 - accuracy: 0.9318 - val_loss: 0.3197 - val_dice_coe: 0.0069 - val_accuracy: 0.9620 - lr: 1.0000e-04
Epoch 2/20


In [ ]:
# Load the best saved model
best_model = tf.keras.models.load_model(best_model_path, custom_objects={"dice_coe": dice_coe})


# Evaluate the model on the validation set
val_loss,dice_coe, val_acc, = model.evaluate(val_dataset)
print("Validation Loss:", val_loss)
print("Validation dice_coe:", dice_coe)
print("Validation Accuracy:", val_acc)


In [ ]:
# Get the loss values from the history object
train_loss = history.history['loss']
val_loss = history.history['val_loss']

# Plot the loss values
epochs = range(1, len(train_loss) + 1)
plt.plot(epochs, train_loss, 'g', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Loss Function')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
num_batches=1
#cutoff = 0.05
val_dataset = val_dataset.take(num_batches)

for batch in val_dataset:
   images, masks = batch
   predicted_masks = best_model.predict(images)
   #predicted_masks = (predicted_masks > cutoff).astype(int)


   print(images.shape,images.dtype,masks.shape,masks.dtype)
   for i in range(images.shape[0]):
            image = images[i]
            mask = masks[i]
            pred = predicted_masks[i]
            print(np.unique(mask.numpy()))
            #ifile = files[i]
            #print(ifile)

            # Plot image and mask side by side
            fig, axes = plt.subplots(1, 3, figsize=(10, 5))
            axes[0].imshow(image)
            axes[0].set_title('Image')
            axes[0].axis('off')
            axes[1].imshow(image)
            axes[1].imshow(mask[:, :, 0], alpha=0.5)
            axes[1].axis('off')
            axes[2].imshow(image)
            axes[2].imshow(pred[:, :, 0], alpha=0.5,vmin=0,vmax=0.1)
            axes[2].axis('off')



            plt.show()


# Test on full size image

In [ ]:
# predicted_masks = best_model.predict(images)
# #predicted_masks

In [ ]:
def make_grid(shape, window=256, min_overlap=32):
    """
        Return Array of size (N,4), where N - number of tiles,
        2nd axis represente slices: x1,x2,y1,y2
    """
    x, y,z = shape
    nx = x // (window - min_overlap) + 1
    x1 = np.linspace(0, x, num=nx, endpoint=False, dtype=np.int64)
    x1[-1] = x - window
    x2 = (x1 + window).clip(0, x)
    ny = y // (window - min_overlap) + 1
    y1 = np.linspace(0, y, num=ny, endpoint=False, dtype=np.int64)
    y1[-1] = y - window
    y2 = (y1 + window).clip(0, y)
    slices = np.zeros((nx,ny, 4), dtype=np.int64)

    for i in range(nx):
        for j in range(ny):
            slices[i,j] = x1[i], x2[i], y1[j], y2[j]
    return slices.reshape(nx*ny,4)

import cv2
import numpy as np
patch_size = 512
NEW_SIZE= 128
scale=patch_size//NEW_SIZE
overlap = 512


filename='1.png'
image_path='/content/drive/MyDrive/Processed_Floor_Plans/FP_raw/'+filename
mask_path='/content/drive/MyDrive/Processed_Floor_Plans/FP_processed/'+filename

image = cv2.imread(image_path)
mask  = cv2.imread(mask_path,cv2.IMREAD_GRAYSCALE)
slices=make_grid(image.shape,window=patch_size,min_overlap=overlap)

image_patches = []
mask_patches = []

slices=make_grid(image.shape,window=patch_size,min_overlap=overlap)
#print(slices)
for i,(x1,x2,y1,y2) in enumerate(slices):
  try:
        image_patch = image[x1:x2,y1:y2]


        #print(image_patch.shape,mask_patch.shape)


        image_patch= cv2.resize(image_patch, (NEW_SIZE, NEW_SIZE),interpolation = cv2.INTER_AREA)
        image_patches.append(image_patch)
  except:
        print(image_path,i)
        pass

image_patches=np.array(image_patches)
image_patches.shape

pred_image=np.zeros_like(mask)
n_pred=np.zeros_like(mask)
predicted_masks = best_model.predict(images)
print(pred_image.shape,predicted_masks.shape)
for i,(x1,x2,y1,y2) in enumerate(slices):
  tmp= cv2.resize(predicted_masks[i]*255, (y2-y1,x2-x1),interpolation = cv2.INTER_AREA)
  pred_image[x1:x2,y1:y2]=tmp
  n_pred[x1:x2,y1:y2]=n_pred[x1:x2,y1:y2]+1

pred_image.shape

plt.imshow(mask)
plt.show()
plt.imshow(pred_image)
plt.show()

In [ ]:
plt.imshow(pred_image)